<a href="https://colab.research.google.com/github/FranTr64/IA-Por-refuerzo/blob/main/Google_Collab/Version_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En este documento se presenta una IA que ha sido entrenada por aprendizaje por refuerzo de forma sencilla con el objetivo de aprender a jugar al tres en raya.

Esta IA ha sido posible gracias a "SensIO".

Video donde he aprendido a hacer esta IA: https://www.youtube.com/watch?v=4Z8WwCYl1CQ&list=PLkgbkukKg_Nr6qZk9GWI1hnmtiCerJHGE.

Su canal de Youtube: https://www.youtube.com/@juansensio

En este video explica los primeros conceptos sobre IA de aprendizaje por refuerzo y los pone a prueba con un ejemplo del juego de tres en raya.

Bastante del codigo está sacado de sus explicaciones, sin embargo, eso no significa que no entienda nada de lo que hay escrito. En cada apartado habrá una explicación mia del objetivo de parte del codigo, con el fin de que se entienda mejor.

In [ ]:
import numpy as np


Primeramente, creamos una clase para el tablero. Esto con el fin de tener un medio donde "poner las fichas". Con el uso de Numpy, crearemos una matriz de tres por tres rellena de ceros, que nos servirá para tener inicializado el tablero y poder poner las "fichas" (que en este caso, serán 1 y -1).

Tendremos varias funciones, comenzando por:

- Movimientos: Aquí obtendremos una lista de todos los movimientos posibles. Con movimientos, se entiende: lugares donde poder poner una ficha.

- Jugada: En esta función, el agente o la persona podrá poner una ficha. Simplemente accederemos a la matriz y sustituiremos el valor el de la ficha (1 o -1, dependiendo de si es el jugador 1 o el jugador 2).

- Show: Esta función tiene como objetivo poder ver la tabla de una forma clara. Si se va a jugar contra la maquina, nosotros necesitamos ver las opciones que hay. Para ello, se transdorma el self.estado en una tabla que nosotros podamos entender, transformando los 1 en X y los -1 en O.

- Fin Juego: Aquí verificaremos si en algun momento alguno de los dos jugadores ha ganado o si ha habido un empate. Se revisará esta función continuamente, y si hay un ganador, se devolverá el numero correspondiente. Si no, enviará None y el juego seguirá.

- Reset: Cuando se llame a esta función, se "limpiará el tablero", que es una forma de decir que sustituiremos self.estado por una matriz de tres por tres rellena de ceros. De esta forma podremos volver a jugar otra ronda sin problema alguno.

In [ ]:
# Creamos una clase para tener un tablero. Esta clase no será la funcion que usaremos para jugar, si no el medio por el que podremos interactuar.
class Tablero():
  def __init__(self):
    self.estado = np.zeros((3,3))

  def movimientos(self):
    return [(fila, columna) for fila in range(3) for columna in range(3) if self.estado[fila, columna] == 0]

  def jugada(self, jugador, fila, columna):
    if self.estado[fila, columna] == 0:
      self.estado[fila, columna] = jugador
    else:
      raise ValueError("Movimiento erroneo. Error")

  def Show(self):
    tablilla = self.estado.copy()
    tablilla = tablilla.tolist()
    for i in range(3):
      for j in range(3):
        if tablilla[i][j] == 1:
          tablilla[i][j] = 'X'
        elif tablilla[i][j] == -1:
          tablilla[i][j] = 'O'
        else:
          tablilla[i][j] = ' '
    tabla = pd.DataFrame(tablilla, columns=['0', '1', '2'])
    print(tabla)

  def fin_juego(self):
    if (self.estado.sum(axis=0) == 3).any() or (self.estado.sum(axis=1) == 3).any() or (np.trace(self.estado) == 3).any() or (np.trace(np.fliplr(self.estado)) == 3).any():
      return 1

    if (self.estado.sum(axis=0) == -3).any() or (self.estado.sum(axis=1) == -3).any() or (np.trace(self.estado) == -3).any() or (np.trace(np.fliplr(self.estado)) == -3).any():
      return -1

    if len(self.movimientos()) == 0:
      return 0

    return None

  def reset(self):
    self.estado = np.zeros((3,3))


En este bloque creamos el Juego, es decir, toda la logica de turnos y cuando dar las recompensas.
Primeramente decidimos el simbolo que tendrá el jugador 1, el jugador 2, asociamos un tablero a este juego y decimos si queremos ver el tablero o no. Esto último porque, si quisieramos entrenar una IA, es mucho mas conveniente que se entrene sin que se esté imprimiendo el tablero una y otra vez.

Para ejecutar la función Partida, primeramente se le ha de decir el numero de rondas a jugar. Si no se le dice, se jugarán 5 por defecto.
Luego establecemos el marcador de victorias [0,0,0]. El primer numero representa al jugador 1, el segundo al jugador 2, y el tercero es el numero de veces que han quedado empatados.

Una vez ajustados estos datos, podemos continuar con el flujo. Iteramos por el numero de rondas. TQDM nos sirve para saber el estado, es decir, cuantas rondas lleva hechas y cuantas le faltan. Sirve a modo de pantalla de carga.

Una vez iniciamos una ronda, establecemos los datos iniciales: restablecemos el tablero a su forma inicial, reseteamos a los jugadores y creamos una variable "game_over" y lo establecemos en falso. Todo esto en la primera ronda no tiene mucho sentido, dado que todo está ya en sus fases iniciales, pero cuando volvamos a comenzar otra ronda, debemos restablecer todo a como estaba antes para poder jugar.

Luego entramos en un buble "while". Basicamente, como hemos instanciado la variable "game_over" en falso desde el inicio, la única forma de salir del bucle es que se convierta en verdadero, y eso ocurrirá cuando alguno de los dos haya ganado o cuando haya un empate.

Iteramos por los jugadores y accedemos a su funcion "Accion". Esta funcion se explicará más adelante, pero a modo de resumen, esta funcion toma el tablero y el jugador analizará la tabla y eligirá el mejor movimiento. Esa será su "movimiento". Luego accedemos a la función "jugadas" del tablero previamente explicada y enviamos, el movimiento del jugador, y su simbolo, que sería su ficha.

Casi en el final, llamamos a la funcion Update de cada jugador y le volvemos a pasar el tablero. "Callable(getattr())" se llama aquí para comprobar si el jugador tiene la funcion Update (que se explicará más adelante). Esto es porque la clase de Jugador Humano es diferente a la clase de Jugador IA, dado que una persona no tiene que crear una base de datos con cada movimiento.

Por último, comprobamos si alguno de los dos ya ha ganado. En los primeros movimientos, es obvio que no se ha podido ganar todavía, pero debido al hecho de que no sabemos cuando cualquiera de los dos puede ganar, tenemos que comprobar a cada rato si ya ha habido una victoria.

En el caso de que hubiera una victoria (es decir, la función no devuelva None), "game_over" pasaría a ser verdadero y se saldría del bucle. En este punto se repartirían las recompensas a cada jugador y se pasaría a la siguiente ronda.

Una vez terminadas las rondas, el juego se acabó y se devuelve la lista de victorias.

La función de Recompensas es lo que le da vida a este tipo de IA de entrenamiento por refuerzo. A la hora de repartir, si la IA ha ganado, se le da 1 punto. Si ha perdido, no se le da nada, y si las dos han quedado empatadas, cada una recibe 0,5. Es de notar que si una recibe 1 punto, la otra recibe 0 y viceversa.

In [ ]:
from tqdm import tqdm
class Juego():
  def __init__(self, jugador_1, jugador_2, vision = False):
    jugador_1.simbolo = 1
    jugador_2.simbolo = -1
    self.jugadores = [jugador_1, jugador_2]
    self.tablero = Tablero()
    self.vision = vision

  def Partida(self, rondas = 5):
    victorias = [0,0,0]
    for ronda in tqdm(range(rondas)):
      self.tablero.reset()
      for jugador in self.jugadores:
        if callable(getattr(jugador, 'Reset', None)):
          jugador.Reset()
      game_over = False
      while not game_over:
        for jugador in self.jugadores:
          if self.vision:
            self.tablero.Show()
          movimiento_jugador = jugador.Accion(self.tablero)
          jugada = self.tablero.jugada(jugador.simbolo, movimiento_jugador[0], movimiento_jugador[1])
          for jugador in self.jugadores:
            if callable(getattr(jugador, 'Update', None)):
              jugador.Update(self.tablero)
          if self.tablero.fin_juego() is not None:
            game_over = True
            break
      self.Recompensa()
      for i, jugador in enumerate(self.jugadores):
        if self.tablero.fin_juego() == jugador.simbolo:
          victorias[i] += 1
      if self.tablero.fin_juego() == 0:
        victorias[2] += 1

    return victorias

  def Recompensa(self):
    if self.tablero.fin_juego() == 0:
      for jugador in self.jugadores:
        if callable(getattr(jugador, 'Recompensa', None)):
          jugador.Recompensa(0.5)
    else:
      for jugador in self.jugadores:
        if callable(getattr(jugador, 'Recompensa', None)):
          if self.tablero.fin_juego() == jugador.simbolo:
            jugador.Recompensa(1)
          else:
            jugador.Recompensa(0)




La clase Agente es lo que le da la vida al Jugador IA.

A la hora de instanciarlo, se le han de poner ciertas variables. Una variable para ver la probabilidad de explorar (de 0 a 1), otra para la variación (se explicará su uso cuando hablemos de las recompensas en breve), y otra para ver si la IA explorará en primer lugar o no. Además, instanciamos su memoria a modo de diccionario y creamos una lista vacía de movimientos.

La función de modificación tiene por objetivo modificar alguna de estas variables previamente instanciadas. Aunque se podría pensar que sería más sencillo crear una nueva instancia, tambien se perdería la memoria y lo que ha aprendido jugada tras jugada. De esta forma, se le pueden hacer modificaciones sin perjudicar a su base de datos.

En la función de Acción, la IA podrá decidir que movimiento hacer. Lo primero de todo será comprobar la lista de movimientos disponibles, con el fin de que no elija una casilla con una ficha ya puesta. Luego comienza la fase de exploración. Si su variable de exploracion es verdadera y la probabilidad que le hemos dado es favorable, la IA eligirá un movimiento aleatorio. Aunque puede parecer indeseable, dado que le estamos dando una memoria por algo, tener una probabilidad de elegir un movimiento al azar le da la oportunidad de descubrir nuevas jugadas que, si se guiará únicamente por su memoria, jamás descubriría.

Si no tuviera habilitada la exploración, o la probabilidad no hubiera resultado favorable, pasaríamos a la fase de explotación. En esta fase, la IA usaría el conocimiento que tiene para poder hacer la mejor jugada posible. Para poder elegir, hace lo siguiente:

- Primeramente iteramos por todos los movimientos de hay disponibles.

- Creamos una copia de la tabla y hacemos una simulacion del movimento que nos toca en esta iteración. Es decir, simulamos que colocamos una ficha en esa posición.

- Miramos la base de datos, y comprobamos si la puntuación que tenemos por realizar ese movimiento es superior al movimiento que tenemos guardado como "la mejor opción" (La puntuación se guarda en "valor", y la posición es "fila_enviar" y "posicion_enviar") (El primer movimiento que se simule siempre será considerado como el mejor). Si no llegara a serlo, se ignora y pasamos al siguiente. Si es superior, guardamos el movimiento (la fila y la columna donde poner la ficha) y pasamos al siguiente.

- Una vez iterado por todos los movimientos, devolvemos el movimiento con mayor puntuación.

Un detalle a notar es que, si no existiera ese movimiento en la base de datos, su puntuación sería de 0. Esto ocurrirá muchas veces al principio, dado que la IA apenas tiene experiencia jugando.

La funcion de reset tiene por objetivo limpiar la lista de movimientos del Agente. Esto con el fin de poder volver a hacer otra partida y no confundir los movimientos de la ronda siguiente con la anterior.

La funcion de Update agrega los movimientos hechos en una lista. Esto será útil en la funcion de Recompensa.

La función de recompensa es la razón por la que la IA de aprendizaje por refuerzo se la llama así. El objetivo es recompensar o castigar los movimientos que ha hecho para que en un futuro las vuelva a repetir, o las repita con mucha menos frecuencia. Se revisan los movimientos, yendo en orden inverso, desde el último hecho hasta el primero y, usando la variable de "variación", y la recompensa obtenida, se le da una puntuación al estado del tablero, lo que podría representar su "movimiento" realizado. Luego, esa misma recompensa será utilizada para valorar el movimiento siguiente, y así sucesivamente. Se guardará todo en la base de datos y lo podrá usar en un futuro. Si el movimiento no existiera en la base de datos, se creará un apartado para ese estado.

Es gracias a esta última función que la IA es capaz de aprender que acciones son buenas y cuales son malas. Cuantas más partidas haga, más veces podrá calibrar sus movimientos y saber que acción es la mejor en cada oportunidad.

In [ ]:
class Agente():
  def __init__(self, variacion, prob_exploracion, exploracion):
    self.variacion = variacion
    self.prob_exploracion = prob_exploracion
    self.memoria = {}
    self.movimientos = []
    self.exploracion = exploracion

  def Modificacion(self, var = None, prob_exp = None, exp = None):
    if var is not None:
      self.variacion = var
    if prob_exp is not None:
      self.prob_exploracion = prob_exp
    if exp is not None:
      self.exploracion = exp

  def Accion(self, tablero):
    movimientos = tablero.movimientos()

    if self.exploracion and np.random.uniform(0, 1) < self.prob_exploracion:
      i = np.random.choice(len(movimientos))
      return movimientos[i]
    decision = -1000
    for fila, columna in movimientos:
      tabla = tablero.estado.copy()
      tabla[fila, columna] = self.simbolo
      estado = str(tabla.reshape(3*3))
      valor = 0 if self.memoria.get(estado) is None else self.memoria.get(estado)
      if valor > decision:
        decision = valor
        fila_enviar, columna_enviar = fila, columna
    return fila_enviar, columna_enviar

  def Reset(self):
    self.movimientos = []

  def Update(self, tablero):
    self.movimientos.append(str(tablero.estado.reshape(3*3)))

  def Recompensa(self, recompensa):
    for movimiento in reversed(self.movimientos):
      if self.memoria.get(movimiento) is None:
        self.memoria[movimiento] = 0
      self.memoria[movimiento] += self.variacion * (recompensa - self.memoria[movimiento])
      recompensa = self.memoria[movimiento]


Este bloque es una prueba rápida para comprobar si todo funciona correctamente. Tiene puesta 50 rondas, pero se les puede poner las que se quieran. Cuantas más rondas se juegue, mayor será el tiempo de entrenamiento.

In [ ]:
jugador1 = Agente(0.5, 0.5, True)
jugador2 = Agente(0.5, 0.5, True)

juego = Juego(jugador1, jugador2)

juego.Partida(50)



100%|██████████| 50/50 [00:00<00:00, 155.46it/s]


[18, 18, 14]

Aquí se puede observar como, verdaderamente, ha aprendido que movimientos son mejores y cuales no.

In [ ]:
import pandas as pd
lista_clave_valor = sorted(jugador1.memoria.items(), key=lambda kv: kv[1], reverse=True)
tabla = pd.DataFrame(lista_clave_valor, columns=['Movimientos', 'Valor_de_movimiento'])

tabla

,Movimientos,Valor_de_movimiento
0,[-1. 1. -1. -1. 1. 1. 0. 1. 0.],0.8750
1,[-1. 1. -1. -1. 0. 1. 0. 1. 0.],0.6875
2,[ 1. 1. -1. -1. 1. -1. 1. -1. 1.],0.5000
3,[ 1. -1. 0. 1. 1. 1. -1. 0. -1.],0.5000
4,[-1. 1. -1. -1. 1. 0. 0. 1. 1.],0.5000
...,...,...
230,[ 1. 1. -1. 0. 1. 0. -1. 0. -1.],0.0000
231,[ 1. 1. 0. 0. 1. 0. -1. 0. -1.],0.0000
232,[ 1. 1. 0. 0. 0. 0. -1. 0. -1.],0.0000
233,[ 1. 1. 0. 0. 0. 0. -1. 0. 0.],0.0000


Está clase está hecha con lo minimo indispensable para que una persona pueda jugar. Tiene la función de Acción y ya. Dado que es una persona la que juega, no necesita de recompensas ni otras variables. Dispone de un Input para que la persona pueda escribir su movimiento.

El jugador tendrá que escribir dos numeros separados de una coma. El primer numero eligirá la fila, y el segundo la columna.

In [ ]:
class Persona():
  def Accion(self, tablero):
    while(True):
      print('vuelta a empezar')
      lista = tablero.movimientos()
      movimiento = input('Elije tu movimiento: ')
      tupla = tuple(map(int, movimiento.replace(" ", "").split(",")))
      if tupla in lista:
        print('hecho')
        return tupla[0], tupla[1]
        break


Aquí se puede hacer una prueba, jugando contra una IA.

In [ ]:
jugador_ia = Agente(0.5, 0.5, True)
persona = Persona()
juego = Juego(jugador_ia, persona, True)

juego.Partida(2)

  0%|          | 0/2 [00:00<?, ?it/s]

   0  1  2
0         
1         
2         
   0  1  2
0         
1  X      
2         
vuelta a empezar
Elije tu movimiento: 2,0
hecho
   0  1  2
0         
1  X      
2  O      
   0  1  2
0     X   
1  X      
2  O      
vuelta a empezar
Elije tu movimiento: 0,0
hecho
   0  1  2
0  O  X   
1  X      
2  O      
   0  1  2
0  O  X   
1  X  X   
2  O      
vuelta a empezar
Elije tu movimiento: 2,1
hecho
   0  1  2
0  O  X   
1  X  X   
2  O  O   
   0  1  2
0  O  X  X
1  X  X   
2  O  O   
vuelta a empezar
Elije tu movimiento: 1,2


 50%|█████     | 1/2 [08:00<08:00, 480.13s/it]

hecho
   0  1  2
0  O  X  X
1  X  X  O
2  O  O   
   0  1  2
0         
1         
2         
   0  1  2
0         
1         
2  X      
vuelta a empezar
Elije tu movimiento: 1,1
hecho
   0  1  2
0         
1     O   
2  X      
   0  1  2
0  X      
1     O   
2  X      
vuelta a empezar
Elije tu movimiento: 0,1
hecho
   0  1  2
0  X  O   
1     O   
2  X      
   0  1  2
0  X  O   
1     O   
2  X     X
vuelta a empezar
Elije tu movimiento: 2,1


100%|██████████| 2/2 [08:26<00:00, 253.01s/it]

hecho


[0, 1, 1]